In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import os 
import earthpy as et 
from os import makedirs

In [2]:
## Code needed to set a new working directory

#my_path = os.path.join(et.io.HOME, 'recommender_system')
#os.mkdir(my_path)

## Set Working Directory
os.chdir(os.path.join(et.io.HOME, 'recommender_system'))

In [3]:
# Check if it worked
os.getcwd()

'/Users/chriskamerin/recommender_system'

In [84]:
df=pd.read_csv('/Users/chriskamerin/Desktop/hp_ratings_urlremoved.csv')

In [85]:
df.head

<bound method NDFrame.head of                                    Product1  Releaseyear1  MixedUsage1  \
0                   Beyerdynamic DT 770 PRO          2020          5.6   
1             Corsair Void PRO RGB Wireless          2020          5.8   
2                   Beats Studio 3 Wireless          2020          7.0   
3                           HiFiMan Sundara          2020          5.2   
4        AfterShokz Aeropex Bone Conduction          2020          5.6   
..                                      ...           ...          ...   
283                 Skullcandy Jib Wireless          2015          6.7   
284   Logitech G933 Wireless Gaming Headset          2015          6.3   
285                Beyerdynamic DT 1770 PRO          2015          5.5   
286  Astro A40 TR Headset + MixAmp Pro 2017          2015          5.5   
287                       HyperX Cloud Core          2015          5.6   

     Commute_Travel1  Sports_Fitness1  Office1  WirelessGaming1  WiredGaming1  \


In [86]:
columns = df.columns.to_list()
new_columns = []
for i in columns:
    new_columns.append(str(i)[:-1])

In [87]:
df[new_columns] = df[columns]

In [88]:
df=df.drop(columns=columns)

In [89]:
df.columns

Index(['Product', 'Releaseyear', 'MixedUsage', 'Commute_Travel',
       'Sports_Fitness', 'Office', 'WirelessGaming', 'WiredGaming',
       'PhoneCalls', 'NeutralSound', 'Type', 'Enclosure', 'Wireless',
       'NoiseCancelling', 'Mic', 'FrequencyResponseConsistency',
       'BassAccuracy', 'MidAccuracy', 'TrebleAccuracy', 'Peaks_Dips',
       'Imaging', 'PassiveSoundstage', 'WeightedHarmonicDistortion',
       'NoiseIsolation', 'Microphone', 'RecordingQuality', 'Bluetooth'],
      dtype='object')

In [90]:
df= df.drop(columns=['Releaseyear','MixedUsage'])

In [91]:
df['game_use'] = df[['WirelessGaming','WiredGaming']].max(axis=1)

In [92]:
df[['game_use','WirelessGaming','WiredGaming']]

,game_use,WirelessGaming,WiredGaming
0,6.5,4.4,6.5
1,7.4,7.4,5.9
2,7.2,5.5,7.2
3,6.7,4.7,6.7
4,4.7,4.7,4.5
...,...,...,...
283,5.0,5.0,5.0
284,7.7,7.3,7.7
285,6.3,4.3,6.3
286,8.0,6.1,8.0


In [93]:
df = df.drop(columns=['WirelessGaming','WiredGaming'])

In [94]:
df=df.reindex(columns=['Product','game_use', 'Commute_Travel', 'Sports_Fitness', 'Office', 'PhoneCalls',
       'NeutralSound', 'Type', 'Enclosure', 'Wireless', 'NoiseCancelling',
       'Mic', 'FrequencyResponseConsistency', 'BassAccuracy', 'MidAccuracy',
       'TrebleAccuracy', 'Peaks_Dips', 'Imaging', 'PassiveSoundstage',
       'WeightedHarmonicDistortion', 'NoiseIsolation', 'Microphone',
       'RecordingQuality', 'Bluetooth'])

In [95]:
new_columns = ['headphone_name','game_use', 'travel_use', 'exercise_use', 'office_use', 'phone_call_use',
       'studio_use', 'type', 'enclosure', 'wireless', 'noise_cancelling',
       'mic_presence', 'frequency_response_consistency', 'bass_accuracy', 'mid_accuracy',
       'treble_accuracy', 'peaks_dips', 'imaging', 'passive_soundstage',
       'weighted_harmonic_distortion', 'noise_isolation', 'microphone_rating',
       'mic_recording_quality', 'bluetooth']
columns = df.columns.to_list()

In [96]:
df[new_columns] = df[columns]

In [97]:
df = df.drop(columns=columns)

In [98]:
df.columns

Index(['headphone_name', 'travel_use', 'exercise_use', 'office_use',
       'phone_call_use', 'studio_use', 'type', 'enclosure', 'wireless',
       'noise_cancelling', 'mic_presence', 'frequency_response_consistency',
       'bass_accuracy', 'mid_accuracy', 'treble_accuracy', 'peaks_dips',
       'imaging', 'passive_soundstage', 'weighted_harmonic_distortion',
       'noise_isolation', 'microphone_rating', 'mic_recording_quality',
       'bluetooth'],
      dtype='object')

In [99]:
df['headphone_name']  = pd.Categorical(df.headphone_name).codes

In [101]:
df['wireless'] = pd.get_dummies(df.wireless, prefix='wireless')[['wireless_Yes','wireless_Truly Wireless']].sum(axis=1)
df['noise_cancelling'] = pd.get_dummies(df.noise_cancelling, prefix='noise_cancelling')['noise_cancelling_Yes']
df['mic_presence'] = pd.get_dummies(df.mic_presence, prefix='mic_presence')['mic_presence_Yes']
df[['closed_back','open_back']] = pd.get_dummies(df.enclosure, prefix='enclosure')[['enclosure_Closed-Back','enclosure_Open-Back']]

In [102]:
df['in_ear']= pd.get_dummies(df.type, prefix='type')[['type_Earbuds','type_In-ear']].sum(axis=1)
df['on_ear'] = pd.get_dummies(df.type, prefix='type')[['type_On-ear']]
df['over_ear'] = pd.get_dummies(df.type, prefix='type')[['type_Over-ear']]
df['head_set'] = pd.get_dummies(df.type, prefix='type')[['type_Mono In-ear','type_Mono On-ear','type_Mono Earbud']].sum(axis=1)

In [104]:
df = df.drop(columns=['type','enclosure'])

In [108]:
columns_to_norm = ['travel_use', 'exercise_use', 'office_use',
       'phone_call_use', 'studio_use','frequency_response_consistency',
       'bass_accuracy', 'mid_accuracy', 'treble_accuracy', 'peaks_dips',
       'imaging', 'passive_soundstage', 'weighted_harmonic_distortion',
       'noise_isolation', 'microphone_rating', 'mic_recording_quality',
       'bluetooth']

In [109]:
df=df.fillna(0)
df[columns_to_norm] = df[columns_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))   


In [111]:
df.to_csv("/Users/chriskamerin/Desktop/headphone_df_cleaned.csv")
